#### Sentiment analysis on tags

I will use the pre-trained model `vaderSentiment`, which comes as a simple pip module

##### Installations

In [2]:
%pip install vaderSentiment


     ------------------------------------ 126.0/126.0 KB 462.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\GiannisAthanasiou\Desktop\diploma\repo\diploma\environ\Scripts\python.exe -m pip install --upgrade pip' command.


In [8]:
# original code from https://towardsdatascience.com/the-most-favorable-pre-trained-sentiment-classifiers-in-python-9107c06442c6

# import the library
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# calculate the negative, positive, neutral and compound scores, plus verbal evaluation
def sentiment_vader(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"

    elif sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"

    else :
        overall_sentiment = "Neutral"
  
    return {
        "negative": negative,
        "neutral": neutral,
        "positive": positive,
        "compound": compound,
        "overall": overall_sentiment,
    }

##### Let's try the analysis on the tags of the MovieLens dataset

In [43]:
from py2neo import Graph
import os
from os.path import join
import pathlib
import pandas as pd
from collections import defaultdict

In [37]:
graph = Graph(
    "bolt://localhost:11005",
    auth=("neo4j", "admin"),
)

In [41]:
def read_csv(filename: str):
    db_directory_path = pathlib.Path(os.getcwd()).parent.absolute()
    data_directory_path = join(db_directory_path, "data")
    file_path = join(data_directory_path, f"{filename}.csv")
    print("Reading from:", file_path)
    df = pd.read_csv(file_path)
    return df

def df_to_json(df):
    return df.to_dict("records")

tags_df = read_csv("tags")
tags = df_to_json(tags_df)

Reading from: c:\Users\GiannisAthanasiou\Desktop\diploma\repo\diploma\movies_db\data\tags.csv


In [47]:
counters = defaultdict(int)

for tag in tags[:10000]:
    overall = sentiment_vader(tag["tag"])["overall"]
    counters[overall] += 1

print(counters)

defaultdict(<class 'int'>, {'Neutral': 6974, 'Positive': 1906, 'Negative': 1120})
